In [14]:
import pandas as pd
import numpy as np
import sys
import tqdm
import os
import glob
from collections import Counter

In [7]:
try:
    import google.colab
    colab = True
    from google.colab import drive
    lda_input = '/content/drive/MyDrive/NLP_files/CRAFT/results/lda/*whole*'
    bertopic_input = '/content/drive/MyDrive/NLP_files/CRAFT/results/bertopic/bertopic_lemmatize_nostopwords_data_2023-01-02_10-04-20.csv'

    embeddings_output_filepath = '/content/drive/MyDrive/NLP_files/CRAFT/data/embeddings/'
    drive.mount('/content/drive')
    device_name = 'cuda'
    sys.path.append('/content/drive/MyDrive/NLP/bert_embeddings')
except ImportError:
    colab = False
    lda_input = '../CRAFT/results/lda/*whole*'
    bertopic_input = '../CRAFT/results/bertopic/bertopic_lemmatize_nostopwords_data_2023-01-02_10-04-20.csv'

    embeddings_output_filepath = '../CRAFT/results/embeddings/'
    device_name = 'cpu'
from emb_helpers import * 


In [4]:
try:
    os.mkdir(embeddings_output_filepath)
except FileExistsError: 
    pass

In [5]:
from transformers import AutoTokenizer, AutoModel

biobert = AutoModel.from_pretrained("dmis-lab/biobert-v1.1").to(device_name)

In [9]:
files = glob.glob(lda_input) + glob.glob(bertopic_input)

In [18]:
Counter(pd.read_csv(files[1])['topic_number']).most_common()

[(0, 19), (-1, 19), (1, 18), (2, 14), (3, 8), (4, 6), (5, 5), (6, 5), (7, 3)]

In [32]:
df = pd.concat([pd.read_csv(file) for file in files])
all_keywords = sum(df['topic_keywords'].apply(eval), [])
all_keywords = list(set([keyword for keyword, prob in all_keywords]))
all_keywords = pd.DataFrame(all_keywords, columns=['keyword'])
embeddings = return_embeddings_for_concepts(all_keywords['keyword'],biobert, 'cpu')
embeddings['keyword'] = all_keywords['keyword']
embeddings = embeddings[['keyword'] + list(range(len(embeddings.columns)-1))]
embeddings.to_csv(os.path.join(embeddings_output_filepath, 'keywords_embeddings.csv'), index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:03<00:00, 29.68it/s]
